<a href="https://colab.research.google.com/github/odesay97/MachineLearing_Class/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이 장에서 배우는 교차검증, 그리드 서치, 랜덤서치는 모델 학습과 관련된 것이 아님
# 검증방식에 대한 것들임을 명심할 것

# 최적화할것임

# 일반적인 모델 생성방법
# 1. 훈련세트로 모델을 생성
#          하이퍼파라미터 조절
# 2. 테스트세트로 모델을 평가
#      성능이 만족스러우면 모델 생성완료
#      성능이 만족스럽지 못하면 위 과정 반복
# => 결과적으로 테스트세트에 맞게 만들어지게 된다

# 따라서 검증세트를 따로 만들기 !
# 6:2:2 정도 비율

# 최적의 값을 찾아보자 ------------------------

# import pandas as pd

# wine = pd.read_csv('https://bit.ly/wine_csv_data')

# data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# target = wine['class'].to_numpy()

# from sklearn.model_selection import train_test_split

# # 테스트 세트를 나눔
# train_input, test_input, train_target, test_target = train_test_split(
#     data, target, test_size=0.2, random_state=42)

# # 테스트 세트에서 검증세트로 나눔
# sub_input, val_input, sub_target, val_target = train_test_split(
#     train_input, train_target, test_size=0.2, random_state=42)

# print(sub_input.shape, val_input.shape)

# target_score = []
# test_score = []

# depth = [3,4,5,6,7,8,9]
# impurity = [0.001,0.0009,0.0008,0.0007,0.0006,0.0005,0.0004,0.0003,0.0002]
# from sklearn.tree import DecisionTreeClassifier

# for a in depth:
#   for b in impurity:

#     dt = DecisionTreeClassifier(random_state=42, max_depth = a,min_impurity_decrease = b)
#     dt.fit(sub_input, sub_target)

#     # print(dt.score(sub_input, sub_target))
#     # print(dt.score(val_input, val_target))
#----------------------------------------------------------

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split

# 테스트 세트를 나눔
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# 테스트 세트에서 검증세트로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 교차검증
# -> 검증세트를 떼어내어 평가하는 과정을 여러번 반복함

# 교차검증 방식으로 평가
#-> 교차검증 사진 검색해보면 바로 알 수 있음
# 보통 5나 10 폴드 교차검증을 사용함
# 기본값은 5
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
# -> 기본적으로 다음과 같이 출력됨
# {'fit_time': array([0.00931716, 0.00749564, 0.00773239, 0.00731683, 0.00710797]), 'score_time': array([0.00109315, 0.00111032, 0.00101209, 0.00106931, 0.00115085]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
# fit_time => 각훈련에서 걸린 시간 -> 위의 데이터는 5번 훈련했으므로 5개 나옴 ( 기본값이 5 )
# test_score => 검증값 ( 가장 중요 ) -> 이 값들의 평균이 현 모델의 검증 점수가 됨 ( 교차검증 )


# --> 교차검증을 하는 것은 좋은데, 먼저 데이터를 잘 섞어줘야함
# 만약 train_test_split()하지 않았다면 분할기를 설정해줘야함 -> StratifiedKFold
import numpy as np

print(np.mean(scores['test_score']))

from sklearn.model_selection import StratifiedKFold

# 교차검증 기본 분할기 설정
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

# 10 폴드 교차검증
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

#-------------------------------------------------------------------------------

# 하이퍼파라미터 튜닝
# 하이퍼파라미터는 모델이 학습할 수없어서 사용자가 지정해야만 하는 파라미터
# -> ex) max_depth, min_samples_split, minimpurity_decrease 등등
# 반대로 모델이 학습하는 파라미터는 모델파라미터
# 튜닝작업은 기본값을 그대로 사용해서 모델을 훈련한 후 , 검증세트의 점수나 교차검증을 통해 최적의 값을 찾는 것

# 그리드 서치
# -> 하이퍼파라미터 탐색과 교차검증을 한번에 수행
# --> 즉, cross_validate()를 호출할 필요 없음( 교차검증을 내가 명시적으로 해줄 필요 없음 )

from sklearn.model_selection import GridSearchCV

# 어떤 하이퍼파라미터를 찾고 싶나? + 어느 값들을?
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# n_jobs은 병렬 실행에서 사용한 cpu 코어의 수 -> -1이면 모든 코어 사용
# cv매개변수의 기본값은 5이므로 paramsㅇ서 설정한 5개의 값들에 대해 5번씩 반복 => 즉, 총 25번 훈련한다.
# => DecisionTreeClassifier(random_state=42) 해당 모델에 대해 그리드 서치
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

# 그리드 서치를 통해 선택된 최적의 하이퍼파라미터 ( 여기서는 min_impurity_decrease의 값) 출력
print(gs.best_params_)
# -> {'min_impurity_decrease': 0.0001}

print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

# 그리드 서치를 이용한 최적화 과정 정리
# 1. 탐색할 매개변수를 지정
# 2. 훈련세트에서


# ----> 그리드 서치 예시

# min_impurity_decrease은 0.0001에서 0.001까지 0.0001씩 증가하도록  [0.0001,0.0002,0.0003, .... , 0.0009]
# max_depth은 ( 5에서 20까지 1씩 증가시켜서 ) [ 5,6,7, ... 19 ]
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

# 랜덤서치
# 싸이파이 사용

from scipy.stats import uniform, randint

# 정수값
rgen = randint(0, 10)
rgen.rvs(10)
# 10번 샘플링 했기 때문에 고르게 샘플링 되는 것 같아 보이지 않음

# 1000번 샘플링
np.unique(rgen.rvs(1000), return_counts=True)

# 실수값
ugen = uniform(0, 1)
ugen.rvs(10)

# min_samples_leaf -> 리프노드가 되기 위한 최소 샘플의 수
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작으면 분할 안됨
# 샘플링 횟수는 n_iter매개변수에 지정
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV

# 랜덤서치의 경우 반복횟수 n_iter 만드시 해줘야함
# 왜? 그리드 서치와 다르게 랜덤 서치의 경우 n_iter로 설정한 값의 범위내에서 랜덤한 값을 뽑는 방식이기 때문
# 즉, 설정하지 않으면 사실상 무한히 하게 됨
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_

print(dt.score(test_input, test_target))


#-----------------------------

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))
